In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [13]:
train_data  = pd.read_csv('/content/drive/MyDrive/competition_data/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/competition_data/test.csv')

결측치 제거 Dataset

In [14]:
train_data_drop = train_data.dropna()

train_drop = train_data_drop.iloc[:, 1:27]
#train_drop = train_drop.drop(['country'], axis=1)
target_drop = train_data_drop.iloc[:, 69]

In [15]:
from sklearn.model_selection import train_test_split
main_train_input, main_test_input, main_train_target, main_test_target = train_test_split(train_drop, target_drop)

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score

결측치 대체(median) Dataset

In [ ]:
'''train_data = train_data.drop(['country'], axis=1)
train_data = train_data.drop(['index'], axis=1)
for col in train_data.columns:
  train_data = train_data.fillna({col:train_data[col].median()})
  target_data = train_data.fillna({col:train_data[col].median()})'''

결측치 대체(KNN) Dataset

In [ ]:
'''from sklearn.impute import KNNImputer

cols = train_data.columns

imputer = KNNImputer(n_neighbors=3)
train_data = imputer.fit_transform(train_data)
train_data = pd.DataFrame(train_data, columns=cols)
target_data = imputer.fit_transform(train_data)
target_data = pd.DataFrame(target_data, columns=cols)

train_data = train_data.astype(int)
target_data = target_data.astype(int)'''

# 왜인지 모르겠지만 결측치 대체하면 모델(ExtraTree) 돌릴때 세션 다운됨

Extra_Tree(GridSearch)

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier

etc_model = ExtraTreesClassifier()
para = {'n_estimators' : range(5, 50, 10), 'max_depth' : range(5, 20), 'max_features' : range(1, 4)}
gs = GridSearchCV(estimator=etc_model, param_grid=para, scoring='accuracy', cv=5, n_jobs=-1)
gs.fit(main_train_input, main_train_target)

print(gs.best_params_)

{'max_depth': 19, 'max_features': 3, 'n_estimators': 45}


In [16]:
etc_model = ExtraTreesClassifier(n_jobs=-1, random_state=42, max_depth=19, max_features=3, n_estimators=45)

etc_model.fit(main_train_input, main_train_target)
pred = etc_model.predict(main_test_input)
print(etc_model.__class__.__name__," : ",accuracy_score(main_test_target,pred))

ExtraTreesClassifier  :  0.7765822784810127


Extra_Tree(Default)  


In [17]:
from sklearn.ensemble import ExtraTreesClassifier
etc_model = ExtraTreesClassifier(n_jobs=-1, random_state=42)

# grid search한게 더 낮음
etc_model.fit(main_train_input, main_train_target)
pred = etc_model.predict(main_test_input)
print(etc_model.__class__.__name__," : ",accuracy_score(main_test_target,pred))

ExtraTreesClassifier  :  0.7803797468354431


앙상블(voting=hard)

In [18]:
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [19]:
svm_model = SVC()
log_model = LogisticRegression()

In [25]:
voting_model = VotingClassifier(
    estimators=[('lr',log_model),('rf',etc_model),('svc',svm_model)], # 3개의 약한 학습기
    voting='hard'
)

for model in (log_model,etc_model,svm_model,voting_model):
  model.fit(main_train_input, main_train_target)
  pred = model.predict(main_test_input)
  print(model.__class__.__name__," : ",accuracy_score(main_test_target,pred))

LogisticRegression  :  0.7205696202531645
ExtraTreesClassifier  :  0.7803797468354431
SVC  :  0.7338607594936709
VotingClassifier  :  0.7386075949367089


앙상블(voting=soft)

In [22]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

xgb_model = XGBClassifier()
rnd_model = RandomForestClassifier()
lgbm_model = LGBMClassifier()

In [26]:
voting_model = VotingClassifier(
    estimators=[('rf',rnd_model),('xgb',xgb_model),('lgbm',lgbm_model)],
    voting = 'soft')

for model in (rnd_model,lgbm_model,xgb_model,voting_model):
  model.fit(main_train_input, main_train_target)
  pred = model.predict(main_test_input)
  print(model.__class__.__name__," : ",accuracy_score(main_test_target,pred))

RandomForestClassifier  :  0.779746835443038
LGBMClassifier  :  0.7518987341772152
XGBClassifier  :  0.729746835443038
VotingClassifier  :  0.7636075949367088


앙상블(voting=soft)  
--> RandomForest, ExtraTree, LGBM

In [28]:
# 내가 한 것 중에 가장 높은 정확도
voting_model = VotingClassifier(
    estimators=[('rf',rnd_model),('ef',etc_model),('lgbm',lgbm_model)],
    voting = 'soft')

for model in (rnd_model,lgbm_model,etc_model,voting_model):
  model.fit(main_train_input, main_train_target)
  pred = model.predict(main_test_input)
  print(model.__class__.__name__," : ",accuracy_score(main_test_target,pred))

RandomForestClassifier  :  0.7772151898734178
LGBMClassifier  :  0.7518987341772152
ExtraTreesClassifier  :  0.7803797468354431
VotingClassifier  :  0.7873417721518987
